# Library

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras import Input
from keras.losses import binary_crossentropy
from keras.layers import Dense, Dropout
from keras.optimizers import SGD
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from keras.callbacks import ModelCheckpoint

2023-10-20 11:42:07.322250: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-10-20 11:42:09.073474: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
xTrain=np.load("../Data/julyData/trainData.npy")[:,:30]
yTrain=np.load("../Data/julyData/trainTarget.npy")
xTest=np.load('../Data/julyData/testData.npy')[:,:30]
yTest=np.load("../Data/julyData/testTarget.npy")
# xTrain_r=np.reshape(xTrain,(xTrain.shape[0],xTrain.shape[1]*2))
# yTrain=np.reshape(yTrain,(yTrain.shape[0]))
# xTest_r=np.reshape(xTest,(xTest.shape[0],xTest.shape[1]*2))
# yTest=np.reshape(yTest,(yTest.shape[0]))
print("X shape: {}\nY shape: {}".format(xTrain.shape,yTrain.shape))

X shape: (1600, 30)
Y shape: (1600, 1)


# PCA

In [ ]:
sc = StandardScaler()
X_train = sc.fit_transform(xTrain_r)
X_test = sc.transform(xTest_r)
pca = PCA(n_components = 100)
X_train = pca.fit_transform(X_train)
print (pca.explained_variance_)
print (pca.explained_variance_ratio_)
print (pca.explained_variance_ratio_.cumsum())
X_test = pca.transform(X_test)
print("X shape: {}".format(X_train.shape))

# NN

In [3]:
def arch():
    model=Sequential()
    model.add(Input(shape=(30)))
    # model.add(Dense(512,activation='relu'))
    # # model.add(Dropout(0.8))
    # model.add(Dense(256,activation='relu'))
    # # model.add(Dropout(0.8))
    # model.add(Dense(128,activation='relu'))
    # # model.add(Dropout(0.8))
    # model.add(Dense(64,activation='relu'))
    # # model.add(Dropout(0.8))
    model.add(Dense(32,activation='relu'))
    # model.add(Dropout(0.8))
    model.add(Dense(16,activation='relu'))
    # model.add(Dropout(0.8))
    model.add(Dense(4,activation='relu'))
    model.add(Dense(1,activation='sigmoid'))
    return model
# arch().summary()


In [4]:
model=arch()
model.summary()
opt=SGD( learning_rate=0.01, momentum=0.9)
model.compile(loss=binary_crossentropy, optimizer='adam', metrics=['accuracy'])
model_path="../Model/NN/{epoch:02d}-{val_accuracy:.4f}.h5"
checkpoint = ModelCheckpoint(model_path, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint]

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 32)                992       
                                                                 
 dense_1 (Dense)             (None, 16)                528       
                                                                 
 dense_2 (Dense)             (None, 4)                 68        
                                                                 
 dense_3 (Dense)             (None, 1)                 5         
                                                                 
Total params: 1593 (6.22 KB)
Trainable params: 1593 (6.22 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


2023-10-20 11:42:34.528249: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1960] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


In [5]:
H=model.fit(xTrain,yTrain,
          validation_data=(xTest, yTest),
          epochs=20,batch_size=1024,
          callbacks=callbacks_list,
          verbose=1)

Epoch 1/20
1/2 [==============>...............] - ETA: 0s - loss: 0.7047 - accuracy: 0.5117
Epoch 1: val_accuracy improved from -inf to 0.49750, saving model to ../Model/NN/01-0.4975.h5
2/2 [==============================] - 1s 212ms/step - loss: 0.7070 - accuracy: 0.5006 - val_loss: 0.7029 - val_accuracy: 0.4975
Epoch 2/20
1/2 [==============>...............] - ETA: 0s - loss: 0.7035 - accuracy: 0.4971
Epoch 2: val_accuracy did not improve from 0.49750
2/2 [==============================] - 0s 65ms/step - loss: 0.7024 - accuracy: 0.5006 - val_loss: 0.6990 - val_accuracy: 0.4975
Epoch 3/20
1/2 [==============>...............] - ETA: 0s - loss: 0.6994 - accuracy: 0.5010
Epoch 3: val_accuracy did not improve from 0.49750
2/2 [==============================] - 0s 69ms/step - loss: 0.6987 - accuracy: 0.5006 - val_loss: 0.6963 - val_accuracy: 0.4975
Epoch 4/20
1/2 [==============>...............] - ETA: 0s - loss: 0.6958 - accuracy: 0.5039

/global/homes/n/nrvora/.conda/envs/readout/lib/python3.8/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(



Epoch 4: val_accuracy did not improve from 0.49750
2/2 [==============================] - 0s 70ms/step - loss: 0.6961 - accuracy: 0.5006 - val_loss: 0.6948 - val_accuracy: 0.4950
Epoch 5/20
1/2 [==============>...............] - ETA: 0s - loss: 0.6931 - accuracy: 0.5186
Epoch 5: val_accuracy did not improve from 0.49750
2/2 [==============================] - 0s 63ms/step - loss: 0.6944 - accuracy: 0.5006 - val_loss: 0.6940 - val_accuracy: 0.4975
Epoch 6/20
1/2 [==============>...............] - ETA: 0s - loss: 0.6951 - accuracy: 0.4717
Epoch 6: val_accuracy improved from 0.49750 to 0.51000, saving model to ../Model/NN/06-0.5100.h5
2/2 [==============================] - 0s 90ms/step - loss: 0.6941 - accuracy: 0.4931 - val_loss: 0.6935 - val_accuracy: 0.5100
Epoch 7/20
1/2 [==============>...............] - ETA: 0s - loss: 0.6930 - accuracy: 0.5156
Epoch 7: val_accuracy did not improve from 0.51000
2/2 [==============================] - 0s 76ms/step - loss: 0.6932 - accuracy: 0.5075 - v

In [ ]:
N = 150
plt.style.use("ggplot")
plt.figure()
plt.plot(np.arange(0, N), H.history["accuracy"], label="train_accuracy")
plt.plot(np.arange(0, N), H.history["val_accuracy"], label="test_accuracy")
plt.title("Qubit State Classification")
plt.xlabel("Epoch #")
plt.ylabel("Accuracy")
plt.legend(loc="lower left")